# Modelling and Analyzing the UAH-DriveSet Using CTHMM

## Introduction

In this notebook, we illustrate how to leverage the **HMMToolkit** for fitting continuous-time hidden Markov models (CTHMM) and performing anomaly detection using the **UAH-DriveSet**. 

In [1]:
using DrWatson
@quickactivate "HMMToolkit"

include(srcdir("HMMToolkit.jl"))
using CSV, DataFrames, Statistics, Distributions, Random, Base.Threads, StatsBase
using .HMMToolkit

The following are some helper functions to prepare accelerometer and GPS data, and to combine the two data sources.  Importantly, to use the **HMMToolkit**, each time series must be identified by a unique **ID**.

In [2]:
## helper functions to prepare accelerometer and gps data, and combine the two data sources
function prepare_accelerometer(folder_name, ID)

    pathname = datadir("UAH-DriveSet", folder_name, "RAW_ACCELEROMETERS.txt")
    df = CSV.read(pathname, DataFrame; delim=' ', header=false)

    df = df[:, 1:8]
    colnames = ["Timestamp", "System_Activated", "X", "Y", "Z", "X_KF", "Y_KF", "Z_KF"]
    rename!(df, colnames)
    df = combine(groupby(df, [:Timestamp]), last)
    sort!(df, [:Timestamp])

    df.time_since = df.Timestamp .- minimum(df.Timestamp)
    df.time_interval = [missing; df.Timestamp[2:end] - df.Timestamp[1:(end-1)]]

    df.ID .= ID
    df.start .= [1; fill(0, nrow(df) - 1)]

    return df

end

function prepare_gps(folder_name, ID)

    pathname = datadir("UAH-DriveSet", folder_name, "RAW_GPS.txt")
    df = CSV.read(pathname, DataFrame; delim=' ', header=false)

    df = df[:, 1:5]
    colnames = ["Timestamp", "Speed", "Latitude", "Longitude", "Altitude"]
    rename!(df, colnames)
    df = combine(groupby(df, [:Timestamp]), last)
    sort!(df, [:Timestamp])

    df.ID .= ID
    df.start .= [1; fill(0, nrow(df) - 1)]

    return df

end

function append_speed(ts_accelerometer, ts_gps)

    idx = [findmin(abs.(ts_accelerometer.Timestamp .- time))[2] for time in ts_gps.Timestamp]
    ts = ts_accelerometer[idx, :]
    ts.Speed .= ts_gps.Speed
    ts = ts[ts.Speed .> 0, :]
    ts.start[1] = 1

    return ts

end

append_speed (generic function with 1 method)

For illustration purposes, we only consider one driver, D2.  Extracting his telematics data:

In [3]:
df_accel = DataFrame()

i = 2  ## only consider driver D2 for illustration purposes
begin
    folder_name = "D" * string(i) * "-Normal1-Secondary"
    df_i = prepare_accelerometer(folder_name, i)
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Normal2-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+5))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+10))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+15))
    append!(df_accel, df_i)


    folder_name = "D" * string(i) * "-Normal-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+20))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+25))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+30))
    append!(df_accel, df_i)
end


df_gps = DataFrame()

i = 2  ## only consider driver D2 for illustration purposes
begin
    folder_name = "D" * string(i) * "-Normal1-Secondary"
    df_i = prepare_gps(folder_name, i)
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Normal2-Secondary"
    df_i = prepare_gps(folder_name, (i+5))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Secondary"
    df_i = prepare_gps(folder_name, (i+10))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Secondary"
    df_i = prepare_gps(folder_name, (i+15))
    append!(df_gps, df_i)


    folder_name = "D" * string(i) * "-Normal-Motorway"
    df_i = prepare_gps(folder_name, (i+20))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Motorway"
    df_i = prepare_gps(folder_name, (i+25))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Motorway"
    df_i = prepare_gps(folder_name, (i+30))
    append!(df_gps, df_i)
end

group_df_accel = groupby(df_accel, :ID)
group_df_gps = groupby(df_gps, :ID)

df_sec = DataFrame()

## combine the two data sources
for i in 1:7

    ts_accelerometer = group_df_accel[i]
    ts_gps = group_df_gps[i]
    ts = append_speed(ts_accelerometer, ts_gps)
    append!(df_sec, ts)

end

group_df = groupby(df_sec, :ID)

for i in 1:length(group_df)
    group_df[i].time_interval = [missing; fill(1, nrow(group_df[i])-1)]
end

## Model Fitting

While there are multiple variables in the dataset, we can specify which responses the model should be fitted on.  Here, we only consider lateral acceleration, longitudinal acceleration and speed.

In [4]:
response_list = ["Y_KF", "Z_KF", "Speed"]
## Y_KF for turning
## Z_KF for acceleration and braking

3-element Vector{String}:
 "Y_KF"
 "Z_KF"
 "Speed"

We assume all trips performed by D2 follow the same model, and will train a CTHMM on them.  For model fitting, we need to first specify the number of latent states in the CTHMM, and prepare the training set.

In [5]:
i = 2  ## only consider driver D2 for illustration purposes

num_state = 20

filter_idx = [element in [i, i+5, i+10, i+15, i+20, i+25, i+30] for element in df_sec.ID]  # train with all trips from driver D2
train_set = df_sec[filter_idx, :]

Row,Timestamp,System_Activated,X,Y,Z,X_KF,Y_KF,Z_KF,time_since,time_interval,ID,start,Speed
,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Int64,Int64,Float64
1,15.85,1,-0.022,-0.009,-0.015,-0.006,-0.003,-0.004,15.25,missing,2,1,75.7
2,16.76,1,-0.009,0.023,-0.01,-0.005,0.014,-0.007,16.16,1.0,2,0,75.8
3,17.76,1,0.012,0.032,0.005,-0.004,0.035,0.001,17.16,1.0,2,0,77.5
4,18.76,1,-0.038,0.011,0.003,-0.019,0.004,-0.006,18.16,1.0,2,0,78.9
5,19.76,1,-0.036,-0.046,0.003,-0.026,-0.029,-0.005,19.16,1.0,2,0,79.3
6,20.77,1,0.017,0.0,0.016,-0.014,-0.017,0.009,20.17,1.0,2,0,80.2
7,21.77,1,0.023,0.014,-0.038,0.021,-0.007,-0.017,21.17,1.0,2,0,80.8
8,22.77,1,0.002,0.03,-0.014,0.017,0.013,-0.008,22.17,1.0,2,0,81.6
9,23.77,1,0.005,-0.009,-0.006,0.007,-0.002,-0.006,23.17,1.0,2,0,81.5


Users can specify their own initial guess for parameters in the CTHMM.  Alternatively, the **HMMToolkit** includes functions to provide initial parameter estimates using the Clustered Method of Moments (CMM).

In [6]:
Random.seed!(1234)  ## for reproducibility
block_size = nothing  ## transition matrix not a block matrix; all states can communicate
model_init = CTHMM_cmm_init(train_set[:, response_list], num_state, ["real", "real", "continuous"];
                                block_size = block_size, n_random = 0)
## specify the data type of each response dimension

π_list_init = model_init.π_list_init
Q_mat_init = model_init.Q_mat_init
state_list_init = vcat(vcat([hcat([model_init.params_init[1][j][2] for j in 1:num_state]...) for d in 1:1]...),
                        vcat([hcat([model_init.params_init[2][j][2] for j in 1:num_state]...) for d in 1:1]...),
                        vcat([hcat([model_init.params_init[3][j][1] for j in 1:num_state]...) for d in 1:1]...))
## taking Normal for both acceleration components and Gamma for speed

3×20 Matrix{Main.HMMToolkit.AnyExpert{s, Main.HMMToolkit.NonZI, Distribution{Univariate, Continuous}} where s<:Main.HMMToolkit.ExpertSupport}:
 NormalExpert{Float64}(0.00222115, 0.0156616)    …  NormalExpert{Float64}(0.00186885, 0.0144923)
 NormalExpert{Float64}(-0.000548077, 0.0164689)     NormalExpert{Float64}(0.0050765, 0.0200232)
 GammaExpert{Float64}(68.1276, 0.979934)            GammaExpert{Float64}(314.624, 0.404994)

Now we are ready for model fitting; printing steps for every 10th iteration:

In [7]:
model_fit = CTHMM_learn_EM(train_set, response_list, Q_mat_init, π_list_init, state_list_init;
                            ϵ = 1e-03, max_iter = 1000, Q_max_iter = 1, soft_decode = 1, print_steps = 10, penalty = false, block_size = block_size)

┌ Info: Iteration 10, updating π_list: 2160.1150273557246 ->  2160.1174145985633, ( + 0.00011051461651301365 % )
└ @ Main.HMMToolkit /Users/ianweng.chan/Documents/Github/HMMToolkit-dev/src/CTHMM-learn-EM.jl:93
┌ Info: Iteration 10 sub 1, updating Q: 2160.1174145985633 ->  2160.6856067702156, ( + 0.026303763296028623 % )
└ @ Main.HMMToolkit /Users/ianweng.chan/Documents/Github/HMMToolkit-dev/src/CTHMM-learn-EM.jl:122
┌ Info: Iteration 10, updating dim 1: 2160.6856067702156 ->  2162.5000464819127, ( + 0.08397518389588027 % )
└ @ Main.HMMToolkit /Users/ianweng.chan/Documents/Github/HMMToolkit-dev/src/CTHMM-learn-EM.jl:147
┌ Info: Iteration 10, updating dim 2: 2162.5000464819127 ->  2163.9818159956594, ( + 0.06852113211083376 % )
└ @ Main.HMMToolkit /Users/ianweng.chan/Documents/Github/HMMToolkit-dev/src/CTHMM-learn-EM.jl:147
┌ Info: Iteration 10, updating dim 3: 2163.9818159956594 ->  2165.849560931116, ( + 0.08631056516512356 % )
└ @ Main.HMMToolkit /Users/ianweng.chan/Documents/Github/H

(Q_mat_fit = [-0.27884430017547274 0.04366032755988335 … 0.0 1.0196677164621633e-48; 0.005079139259224042 -0.2570361156511108 … 0.0 5.379220949880284e-94; … ; 0.0 0.0 … -0.04408784564513961 1.3640923552261086e-122; 1.0 1.0 … 1.0 -19.0], π_list_fit = [5.319238599769318e-260, 8.208792567647498e-73, 1.5193563015760496e-90, 0.0, 0.0, 4.960481338155335e-217, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.3354414547201743e-159, 0.0, 0.0], state_list_fit = Main.HMMToolkit.AnyExpert{s, Main.HMMToolkit.NonZI, Distribution{Univariate, Continuous}} where s<:Main.HMMToolkit.ExpertSupport[NormalExpert{Float64}(-0.0017538322191561788, 0.04271965418488566) NormalExpert{Float64}(0.0005982073268413104, 0.024722276798514745) … NormalExpert{Float64}(-0.0012609858594920683, 0.026432166675035396) NormalExpert{Float64}(-0.009, 1.0e-5); NormalExpert{Float64}(-0.02733752562993106, 0.07625073073512241) NormalExpert{Float64}(0.008781610707965426, 0.039230404118010806) … NormalExpert{Float64}(-0.006590

And the estimation algorithm converged in 418 iterations.

In [8]:
model_fit.converge, model_fit.iter

(true, 418)

## Unsupervised Anomaly Detection

Having fitted a CTHMM to the 7 trips of D2, we can perform unsupervised anomaly detection to identify outliers and anomalous driving behaviour.  We compute the normal forecast pseudo-residuals of each trip under the fitted model.

In [9]:

ordinary_list, forecast_list = CTHMM_batch_pseudo_residuals(0, train_set, response_list, model_fit.Q_mat_fit, model_fit.π_list_fit, model_fit.state_list_fit;
                                                                keep_last = 1, ZI_list = nothing)
## compute normal pseudo-residuals

(ordinary_list = Array{Float64}[[-0.3718908485179153 -1.6350619804997333 0.20233075735640862; 0.5172715051864387 -0.28630489268665993 -0.30914378647848667; … ; 0.10808998586608957 -0.5189992955363303 -0.7003711435178072; -0.2814333119715584 0.8730719181949269 -1.7866361770075545], [0.848374312806402 0.7189653381807702 -0.6184311926640793; 1.7674595817607535 2.145113703203488 -0.651842117117887; … ; 0.001411431345156829 -0.9582618309511277 -0.06548213727063527; 0.4819405537656469 0.22487050828098673 -1.075092767894046], [-0.08716231087557465 1.0030720835387619 1.240592092857531; 0.6738444897621011 1.2080620255254646 -1.6753286130676301; … ; 0.14108192851911008 -0.003574270794863398 -0.9234274742599627; -0.08863964965289102 0.8048945704412248 -2.15559753536697], [0.8890498181838792 0.0695784916482172 1.0864409452641628; 1.1618763427785557 -0.1267870350962376 -0.1385634920643297; … ; 0.7250001905912016 -0.3241517059674107 -0.31675709743504643; 0.2610527588857224 0.11978916502025573 -1.095

We can assess the forecast pseudo-residuals at each data point.

In [10]:
forecast_list[1]

639×3 Matrix{Float64}:
  0.470239    -0.156887   -0.0670081
  1.20973     -0.0392371   1.00676
  0.143166    -0.304741    0.357352
 -1.17403     -0.334749    0.0556223
 -0.70318      0.0419139   0.37396
 -0.306273    -0.661034    0.559367
  0.48937     -0.419645    0.740123
 -0.114553    -0.36888     0.444507
  0.0815133   -0.45783     0.515496
 -0.932063     0.0535563  -0.586115
  ⋮                       
  0.0803711   -0.256556   -0.719002
 -0.206965     0.115699   -0.878994
 -0.375572    -0.0514979  -0.537832
 -0.00348111   0.0322842  -0.112203
  0.374744    -0.401114    0.170077
  0.0392072   -0.266452    0.169677
  0.0399488    0.148369   -0.146665
  0.104251    -0.54715    -0.738732
 -0.281433     0.873072   -1.78664

We can also quantify the level of anomalousness of each trip using the anomaly indices (outlier proportions).

In [11]:
CTHMM_anomaly_indices(forecast_list)

7×3 Matrix{Float64}:
 0.00312989  0.0         0.0
 0.00442478  0.0         0.0
 0.0         0.0214876   0.00165289
 0.0205047   0.00473186  0.0
 0.00321888  0.00214592  0.00107296
 0.00937866  0.0199297   0.00117233
 0.011879    0.00431965  0.0

The two aggressive trips are characterized by their highest anomaly indices in longitudinal acceleration (2nd column), while the two drowsy trips are chracterized by their highest anomaly indices in lateral acceleration (1st column).

Alternatively, users can compute the anomaly indices directly from the fitted CTHMM:

In [12]:
CTHMM_batch_anomaly_indices(train_set, response_list, model_fit.Q_mat_fit, model_fit.π_list_fit, model_fit.state_list_fit;
                                keep_last = 1, ZI_list = nothing)

7×3 Matrix{Float64}:
 0.00312989  0.0         0.0
 0.00442478  0.0         0.0
 0.0         0.0214876   0.00165289
 0.0205047   0.00473186  0.0
 0.00321888  0.00214592  0.00107296
 0.00937866  0.0199297   0.00117233
 0.011879    0.00431965  0.0